In [111]:
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from termcolor import colored
import wandb
import importlib
import re

import dataset_handler as dh
import loading_utils as lu
import testing_utils as tu

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

#model_name = "gpt-j"
model_name = "codegen"

In [3]:
transformers.set_seed(5)
if model_name == "gpt-j":
    """GPT-J and codeparrot models run in HFTest venv"""
    tokenizer = AutoTokenizer.from_pretrained(gptj_model)
    model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()
elif model_name == "codegen":
    """CodeGen runs in the venv venv"""
    model_args = lu.model_args()
    #model_args.model = "codegen-350M-mono"
    model, tokenizer = lu.load_CodeGen(model_args)

loading parameters
loading parameters took 470.61s
loading tokenizer
loading tokenizer took 3.98s


In [4]:
#import exp_impl.eq_legacy.func_def_eq_short as exp_impl
import exp_impl.func_def_general as exp_impl


#priming_text_path = "data/priming_texts/gsm8k/clustering_prompt/3_clusters/cluster_2.txt"  # for codegen
#priming_text_path = "data/priming_texts/gsm8k/codegen/func_eq_short.txt"
priming_text_path = "data/priming_texts/gsm8k/codegen/func_short.txt"
#priming_text_path = "data/priming_texts/gsm8k/concepts_prompt/part-whole_3.txt"
#wandb_run_name = "@100-codegen-0"
importlib.reload(exp_impl)

"""Load gsm8k"""

if model_name == "gpt-j":
    priming_text_path = (
        "data/priming_texts/gsm8k/gpt-j/gsm8k_fewer_alt.txt"  # for gpt-j
    )
    current_dataset = dh.init_dataset_from_name(
        "gsm8k", primingtext_path=priming_text_path
    )
else:
    current_dataset = dh.init_dataset_from_name(
        "gsm8k",
        primingtext_path=priming_text_path,
        sample_func=exp_impl.sample_n_for_prompting,
        generate_prompt_func=exp_impl.generate_prompt,
    )

tu.set_all_seeds()
#tu.set_all_seeds_alt()

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(100, inc_eq=False)

with open("test_prompt_gen.txt", "w") as f:
    f.write(current_dataset.generate_prompt(sample_q_list[0]))

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

def exercise4():
    """
    Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days?
    """
80


In [138]:
# Set up for CodeGen
config = lu.codegen_gen_args()
#config.num_return_sequences = 4 # 4 for gsm8k 5 for asdiv
config.num_return_sequences = 1
config.k = 3
config.max_length_after_input = 30
#config.top_p = 0.95
config.top_p = 0.7
config.top_k = 10
#config.temperature = 0.7
config.temperature = 1
config.min_length = 1

gen_args = config


In [165]:
def preproc_gen_toks(gen_toks, input_len, tokenizer):
    print(len(gen_toks[0]))
    for gen_tok in gen_toks:
        last_tokens = gen_tok[input_len:]
        generated_text = tokenizer.decode(last_tokens)

    return generated_text.split("\n")[1]

In [172]:
transformers.set_seed(5)
prompt = current_dataset.generate_prompt(sample_q_list[2])

In [179]:
tokens = tokenizer(prompt, return_tensors="pt").input_ids
generated_tokens = model.generate(
    tokens.long().cuda(),
    use_cache=True,
    do_sample=False,
    top_k=gen_args.top_k,
    temperature=gen_args.temperature,
    top_p=gen_args.top_p,
    min_length=len(tokens[0]) + gen_args.min_length,
    max_length=len(tokens[0]) + gen_args.max_length_after_input,
    num_return_sequences=gen_args.num_return_sequences,
    pad_token_id=tokenizer.eos_token_id,
)

list_outputs = preproc_gen_toks(
    generated_tokens, len(tokens[0]), tokenizer
)
print(len(tokens[0]))
print(len(tokens[0]) + gen_args.max_length_after_input)
print(list_outputs)

566
536
566
    return float(cars_per_level_more)


In [178]:
prompt += "\n" + list_outputs
print(prompt)

def exercise1():
    """
    To run his grocery store, Mr. Haj needs $4000 a day. This money is used to pay for orders done, delivery costs and employees' salaries. If he spends 2/5 of the total operation costs on employees' salary and 1/4 of the remaining amount on delivery costs, how much money he pays for the orders done?
    """
    cost_per_day = 4000
    employees_cost = cost_per_day * 2 / 5
    delivery_costs = (cost_per_day - employees_cost) * 1 / 4
    orders_costs = cost_per_day - delivery_costs - employees_cost
    return float(orders_costs)


def exercise2():
    """
    Sue works in a factory and every 30 minutes, a machine she oversees produces 30 cans of soda. How many cans of soda can one machine produce in 8 hours?
    """
    cans_per_30_minutes = 30
    cans_per_minute = cans_per_30_minutes / 30
    cans_per_hour = cans_per_minute * 60
    cans_per_8_hours = cans_per_hour * 8
    return float(cans_per_8_hours)


def exercise3():
    """
    Mr. Rainwater has some goa